In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
color_pal = sns.color_palette()
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



2025-05-26 09:19:36,813 - __main__ - INFO - 📚 All imports loaded successfully
2025-05-26 09:19:36,815 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-26 09:19:36,816 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00
2025-05-26 09:19:36,816 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, target range: 2025-03-15 00:00:00+00:00 → 2025-03-21 23:00:00+00:00
2025-05-26 09:19:36,851 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen
2025-05-26 09:19:37,038 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-26 09:19:37,039 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekd

In [4]:
import sqlite3

conn = sqlite3.connect('../data/WARP.db')
df = pd.read_sql_query("SELECT * FROM training_set", conn)
conn.close()
# change datetime to index
df.set_index('target_datetime', inplace=True)
# convert to datetime
df.index = pd.to_datetime(df.index)
print(df.dtypes)


Price                       float64
Load                        float64
shortwave_radiation         float64
temperature_2m              float64
direct_normal_irradiance    float64
diffuse_radiation           float64
Flow_NO                     float64
yearday_cos                 float64
Flow_GB                     float64
month                         int64
is_dst                        int64
yearday_sin                 float64
is_non_working_day            int64
hour_cos                    float64
is_weekend                    int64
cloud_cover                 float64
weekday_sin                 float64
hour_sin                    float64
weekday_cos                 float64
hour                        float64
day_of_week                 float64
day_of_year                 float64
local_datetime               object
is_holiday                  float64
run_date                     object
wind_speed_10m              float64
wind_direction_10m          float64
snowfall                    

In [ ]:
# save df to CSV


In [ ]:
df['Price'].plot(kind='hist', bins=500)
plt.ylim(top=120)
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Price Distribution')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import timedelta

# Feature and target setup
features = [
    'apparent_temperature',
    'temperature_2m',
    'direct_normal_irradiance',
    'diffuse_radiation',
    'yearday_sin',
    'hour_sin',
    'is_non_working_day',
    'is_dst',
    'is_weekend',
    'is_holiday',
    'weekday_cos',
    'wind_speed_10m',
    'hour_cos',
    'weekday_sin',
    'cloud_cover',
    'yearday_cos' 
] # weather and time features only
target = 'Price'

# Safe datetime handling
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')
    df = df.set_index('datetime')
else:
    print("'datetime' column not found in columns. Sorting by index instead.")
    df = df.sort_index()

# Forecast settings
start_date = pd.Timestamp("2025-03-15 00:00", tz='UTC')
end_date = pd.Timestamp("2025-05-22 00:00", tz='UTC')
lag = timedelta(hours=24)
forecast_horizon = timedelta(hours=6*24) # 6 days

# Store RMSEs
rmses = []

current_time = start_date

train_data = df[df.index < current_time]
test_start = current_time + lag  # check current time  
test_end = test_start + forecast_horizon
test_data = df[(df.index >= test_start) & (df.index < test_end)]

if test_data.empty:
    print(f"No test data for forecast starting at {current_time}")
else:
    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    # Train and predict
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmses.append(rmse)

    # Plot the true vs predicted values for the forecast horizon
    plt.figure(figsize=(15, 5))
    plt.plot(y_test.index, y_test.values, label='True Price', color=color_pal[0])
    plt.plot(y_test.index, y_pred, label='Predicted Price', color=color_pal[1])
    plt.title('True vs Predicted Price (Single Forecast Horizon)')
    plt.xlabel('Datetime')
    plt.ylabel('Price')
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import datetime

# Configuration
MODEL_CODE = "XGB_1fold_split"
PARAMETERS_USED = {
    "temp_2m": True,
    "shortwave_radiation": True,
    "windspeed_10m": True
}
timestamp_predict = pd.to_datetime("2025-03-15 00:00:00", utc=True)
TARGET = 'Price'

# Define features
COMMON_FEATURES = [
    'yearday_cos', 'yearday_sin', 'month',
    'shortwave_radiation', 'windspeed_10m', 'apparent_temperature', 'temperature_2m',
    'direct_normal_irradiance', 'diffuse_radiation',
    'cloud_cover', , 'hour_cos', 'hour_sin', 'is_non_working_day',
    'weekday_sin', 'weekday_cos', 'is_holiday',
]

# Add Load only to training features
TRAIN_FEATURES = COMMON_FEATURES + ['Load', 'Flow_NO', 'Flow_GB', 'Flow_BE', 'Wind_Vol', 'Solar_Vol']
TEST_FEATURES = COMMON_FEATURES

# Sort by index
df = df.sort_index()

# Train-test split based on timestamp_predict
train = df[df.index < timestamp_predict]
test = df[df.index >= timestamp_predict]

X_train = train[TRAIN_FEATURES]
y_train = train[TARGET]
X_test = test[TEST_FEATURES]
y_test = test[TARGET]

# Train model
reg = xgb.XGBRegressor(
    base_score=0.5,
    booster='gbtree',
    n_estimators=1200,
    early_stopping_rounds=50,
    objective='reg:squarederror',
    max_depth=3,
    learning_rate=0.02
)

reg.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=100
)

y_pred = reg.predict(X_test)

# Store predictions
y_test.index = pd.to_datetime(y_test.index)
pred_df = pd.DataFrame({
    'datetime': y_test.index,
    'y_true': y_test.values,
    'y_pred': y_pred
})

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"1-Fold RMSE: {rmse:.4f}")

# Collect results
results = []
for _, row in pred_df.iterrows():
    results.append({
        'timestamp_predict': timestamp_predict.strftime('%d-%m-%Y %H:%M'),
        'datetime': row['datetime'].strftime('%d-%m-%Y %H:%M'),
        'model_code': MODEL_CODE,
        'price': round(row['y_pred'], 4),
        'true_price': round(row['y_true'], 4),
        'RSME': round(rmse, 4),
        'fold': 1,
        'parameters(JSON)': json.dumps(PARAMETERS_USED)
    })

# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("model_results_log.csv", mode='a', index=False)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import timedelta

# Feature and target setup
features = ['hour_cos', 'Load', 'hour_sin', 'weekday_sin', 'weekday_cos', 'Solar_Vol', 'Wind_Vol',
            'WindOffshore_Vol', 'is_holiday','Total_Flow']

target = 'Price'

# Safe datetime handling
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')
    df = df.set_index('datetime')
else:
    print("'datetime' column not found in columns. Sorting by index instead.")
    df = df.sort_index()

# Forecast settings
start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-05 12:00", tz='UTC')
lag = timedelta(hours=36)
forecast_horizon = timedelta(hours=144)

# Store RMSEs
rmses = []

current_time = start_date
while current_time <= end_date:
    train_data = df[df.index < current_time]
    test_start = current_time + lag
    test_end = test_start + forecast_horizon
    test_data = df[(df.index >= test_start) & (df.index < test_end)]

    if test_data.empty:
        print(f"No test data for forecast starting at {current_time}")
        current_time += timedelta(days=1)
        continue

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    # Train and predict
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmses.append(rmse)

    print(f"Forecast origin: {current_time}, Predicting {test_start} to {test_end}, RMSE: {rmse:.3f}")

    current_time += timedelta(days=1)

# Summary
avg_rmse = np.mean(rmses)
print(f"\nAverage RMSE over {len(rmses)} runs: {avg_rmse:.3f}")

In [ ]:
# Extract the forecast origin dates and RMSEs from the previous loop
forecast_origins = []
rmses_per_run = []

start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-05 12:00", tz='UTC')
num_runs = len(rmses)
current_time = start_date

for i in range(num_runs):
    forecast_origins.append(current_time)
    current_time += timedelta(days=1)

# Plot RMSE vs. forecast origin date
plt.figure(figsize=(15, 5))
plt.plot(forecast_origins, rmses, marker='o', linestyle='-', color=color_pal[0])
plt.title('RMSE vs. First Predicted Date (per run)')
plt.xlabel('Forecast Origin (First Predicted Date)')
plt.ylabel('RMSE')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Calculate RMSE per day (averaged over all runs where the day is included in the forecast horizon)

# Assume forecast_origins and rmses are available from previous cells
# Each run predicts 6 days (144 hours), so for each run, map RMSE to each predicted day

# Build a DataFrame mapping each forecast run to its predicted days
forecast_horizon_days = 6
forecast_origin_dates = pd.to_datetime(forecast_origins)
rmse_per_day = {}

for run_idx, origin in enumerate(forecast_origin_dates):
    for day_offset in range(forecast_horizon_days):
        day = (origin + pd.Timedelta(hours=36) + pd.Timedelta(days=day_offset)).normalize()
        if day not in rmse_per_day:
            rmse_per_day[day] = []
        rmse_per_day[day].append(rmses[run_idx])

# Compute average RMSE per day
avg_rmse_per_day = pd.Series({day: np.mean(vals) for day, vals in rmse_per_day.items()})
avg_rmse_per_day = avg_rmse_per_day.sort_index()

# Plot
plt.figure(figsize=(15, 5))
plt.plot(avg_rmse_per_day.index, avg_rmse_per_day.values, marker='o', linestyle='-', color=color_pal[1])
plt.title('Average RMSE per Predicted Day (Averaged over all runs)')
plt.xlabel('Predicted Day')
plt.ylabel('Average RMSE')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Get feature importances from the last trained model
importances = model.feature_importances_
feature_names = model.feature_names_in_

# Create a DataFrame for better visualization
feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feat_imp_df)

# Optional: Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df, palette='viridis')
plt.title('Feature Importance (XGBoost)')
plt.tight_layout()
plt.show()